# Deep Learning for text: sequence-to-sequence learning
In this notebook we will implement a sequence-to-sequence model. This model takes an input sequence and returns another sequence. A sequence-to-sequence model is behind many machine learning tasks beyond classification such as: machine translation, text summarization, question answering, chatbots, and text generation. The sequence-to-sequence model has two components: an encoder and a decoder. The encoder learns from the training set a representation that is useful for the task at hand. In machine translation the encoder maps equivalent terms found in a text in two vocabularies and the correct position in the target sequence. The decoder maps the representation into text in the target language. There are two approaches to build a sequence-to-sequence model: 1) using RNN encoder and decoder, such as LSTM layers, and 2) using a transformer encoder and decoder. A RNN has some drawbacks, a major one is that it cannot handle very long sequences. The transformer architecture was built to perform machine translation and can handle long sequences. In this notebook we will use an encoder and decoder transformer to build to translate a text from English to Spanish. 

![Transformer encoder-decoder](images/transformer_encoder_decoder.jpg)

In [ ]:
!pip install tensorflow==2.8

In [ ]:
import tensorflow as tf
import keras
print('Tensorflow version: {}'.format(tf.__version__))
print('Keras version: {}'.format(keras.__version__))